Gender Net : https://www.dropbox.com/s/iyv483wz7ztr9gh/gender_net.caffemodel?dl=0"

Age Net : https://www.dropbox.com/s/xfb20y596869vbb/age_net.caffemodel?dl=0"

In [ ]:
import cv2
import math
import time

In [5]:
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn, bboxes


faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Load network
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)

frame = cv2.imread("sample1.jpg")
frameFace, bboxes = getFaceBox(faceNet, frame)
padding = 10
for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imwrite("age-gender-out.jpg",frameFace)

Gender : Male, conf = 1.000
Age Output : [[5.0422677e-05 6.6032046e-03 9.9196303e-01 9.0643363e-05 1.2083673e-03
  4.4209090e-05 2.6494012e-05 1.3531237e-05]]
Age : (8-12), conf = 0.992
Gender : Male, conf = 1.000
Age Output : [[9.8923862e-01 1.0366707e-02 3.6442763e-04 2.6064206e-06 1.5998328e-05
  7.6123233e-06 1.7588355e-06 2.2897334e-06]]
Age : (0-2), conf = 0.989
Gender : Female, conf = 0.950
Age Output : [[1.65868153e-06 1.00786008e-06 1.11874564e-04 3.56943841e-04
  9.99294400e-01 2.22708652e-04 9.56830445e-06 1.78582150e-06]]
Age : (25-32), conf = 0.999


In [12]:
frame = cv2.imread("../2020.01.30/me.jpg")
frameFace, bboxes = getFaceBox(faceNet, frame)
padding = 10
for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imwrite("my_age_gender_out.jpg",frameFace)

Gender : Male, conf = 1.000
Age Output : [[2.2212009e-05 1.6818031e-04 7.0358872e-02 1.4526410e-01 7.7968121e-01
  4.3995227e-03 4.2268814e-05 6.3608481e-05]]
Age : (25-32), conf = 0.780


In [13]:
frame = cv2.imread("transgender.jpg")
frameFace, bboxes = getFaceBox(faceNet, frame)
padding = 10
for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imwrite("transgender_out.jpg",frameFace)

Gender : Male, conf = 0.625
Age Output : [[1.5666639e-04 3.2670572e-03 6.3890028e-01 3.2811189e-01 2.8241705e-02
  1.0853987e-03 1.4698762e-04 8.9922192e-05]]
Age : (8-12), conf = 0.639


In [14]:
frame = cv2.imread("topgay.jpg")
frameFace, bboxes = getFaceBox(faceNet, frame)
padding = 10
for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imwrite("topgay_out.jpg",frameFace)

Gender : Male, conf = 1.000
Age Output : [[0.00413423 0.0095104  0.05485668 0.34953934 0.39963442 0.17221673
  0.00535634 0.00475186]]
Age : (25-32), conf = 0.400


In [9]:
frame = cv2.imread("us.jpg")
frameFace, bboxes = getFaceBox(faceNet, frame)
padding = 10
for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imwrite("us_out.jpg",frameFace)

Gender : Female, conf = 0.923
Age Output : [[3.1624702e-05 1.8959057e-03 3.5576898e-01 5.7390803e-01 6.7957647e-02
  3.2722126e-04 2.3635761e-05 8.6957290e-05]]
Age : (15-20), conf = 0.574
Gender : Male, conf = 1.000
Age Output : [[9.9135279e-05 5.8468403e-03 8.8790774e-01 9.2454009e-02 6.1920481e-03
  6.8361806e-03 5.9983571e-04 6.4146057e-05]]
Age : (8-12), conf = 0.888


In [16]:
frame = cv2.imread("dog1.jpg")
frameFace, bboxes = getFaceBox(faceNet, frame)
padding = 10
for bbox in bboxes:
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        genderNet.setInput(blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))

        ageNet.setInput(blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        print("Age Output : {}".format(agePreds))
        print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))

        label = "{},{}".format(gender, age)
        cv2.putText(frameFace, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)        
        cv2.imwrite("dog1_out.jpg",frameFace)

Gender : Male, conf = 1.000
Age Output : [[4.2230054e-06 8.0636775e-05 4.7435886e-01 4.9757490e-01 2.7501499e-02
  4.6555290e-04 9.4733168e-06 4.8220195e-06]]
Age : (15-20), conf = 0.498
Gender : Female, conf = 0.998
Age Output : [[5.5682554e-04 5.5498518e-03 2.5843915e-01 4.5859802e-01 2.7030435e-01
  5.2802037e-03 1.0982582e-03 1.7327804e-04]]
Age : (15-20), conf = 0.459
